# Imports

In [2]:
import os
import json

import gandula

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Pre-processing

In [3]:
shot_frames = gandula.loader.read_pickle(
    'data/enhaced_frames_shots.pkl'
)

In [14]:
result = set()
# iterate over all frames to discover what event features are useful
for frame in shot_frames:
    event_features = list(frame['event'].shootingEvent.model_dump(
        exclude_none=True
    ).keys())
    result.update(event_features)

result

{'blockerPlayer',
 'clearerPlayer',
 'createsSpace',
 'deflectorBodyType',
 'deflectorPlayer',
 'keeperTouchType',
 'pressurePlayer',
 'pressureType',
 'saveHeightType',
 'saveReboundType',
 'saverPlayer',
 'shooterPlayer',
 'shotBodyType',
 'shotInitialHeightType',
 'shotNatureType',
 'shotOutcomeType',
 'shotPointX',
 'shotPointY'}

In [ ]:
def extract_features(raw_frames):
    """
    Extract features from raw data and returns the pandas DataFrame
    """
    data = []

    for frame in raw_frames:
        attacking_team = 

        # extract event features
        event_id            = frame['frame'].event_id
        frame_id            = frame['frame'].frame_id
        x_shot              = frame['event'].shootingEvent.shotPointX
        y_shot              = frame['event'].shootingEvent.shotPointY
        outcome             = frame['event'].shootingEvent.shotOutcomeType
        body_part           = frame['event'].shootingEvent.shotBodyType
        save_height         = frame['event'].shootingEvent.saveHeightType
        shot_initial_height = frame['event'].shootingEvent.shotInitialHeightType
        shooter_id          = frame['event'].shootingEvent.shooterPlayer.id
        

        # extract tracking features





# EDA